## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import folium
import glob

## Notes

Collar counties: DuPage, Will, Kane, Lake and McHenry and is meant to compare the preceeding complete months

Property types: Single Family, Condos, Co-Ops

## Data Read-in

In [2]:
# Find all CSV files in the current working directory
csv_files = glob.glob('*.csv')

# Concatenate all CSV files into a single dataframe
df = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)

In [3]:
## Ensure that no file is over 10K entries ##
[len(pd.read_csv(x)) for x in csv_files]

[1825, 1400, 3306, 2545, 3558, 2591]

## Data Clean

In [4]:
df = df.loc[df['SALE TYPE'] != 'Over 500 results. Try zooming in or modifying your search options.']
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})
df = df.dropna(subset=['SOLD DATE'])

In [5]:
# Define list of desired months (excluding current month)
desired_months = ['May']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Whittle down the property types to only Single Family Residential, Condo/Co-op and Townhouse
df_filtered = df_filtered[(df_filtered['PROPERTY TYPE'] == 'Condo/Co-op') | (df_filtered['PROPERTY TYPE'] == 'Single Family Residential') | (df_filtered['PROPERTY TYPE'] == 'Townhouse')]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [6]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    3558
Name: count, dtype: int64
-------
$/SQUARE FEET
False    3471
True       87
Name: count, dtype: int64
-------
YEAR BUILT
False    3485
True       73
Name: count, dtype: int64
-------


In [7]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LATITUDE,LONGITUDE,Unnamed: 0,URL,PRICE_AS_CURRENCY,YEAR BUILT DISPLAY,PRICE_SQUARE_FEET_AS_CURRENCY,COLOR,RANK,FULL_ADDRESS


In [8]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [9]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LATITUDE,LONGITUDE,Unnamed: 0,URL,PRICE_AS_CURRENCY,YEAR BUILT DISPLAY,PRICE_SQUARE_FEET_AS_CURRENCY,COLOR,RANK,FULL_ADDRESS
3390,PAST SALE,May-31-2023,Single Family Residential,45 Dinosaur Rd,Wilmington,IL,60481.0,29999.0,2.0,1.0,...,41.288945,-88.206888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2771,PAST SALE,May-25-2023,Single Family Residential,2229 Winnebago Rd,Waukegan,IL,60087.0,30000.0,2.0,1.0,...,42.393640,-87.873598,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2238,PAST SALE,May-31-2023,Condo/Co-op,16 Saint Thomas Colony #4,Fox Lake,IL,60020.0,42500.0,1.0,1.0,...,42.428887,-88.196311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2180,PAST SALE,May-11-2023,Condo/Co-op,22 Jamaica Colony #4,Fox Lake,IL,60020.0,50000.0,1.0,1.0,...,42.428361,-88.197072,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2421,PAST SALE,May-17-2023,Condo/Co-op,45 Nassau Colony #7,Fox Lake,IL,60020.0,52000.0,0.0,1.0,...,42.427010,-88.197589,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2692,PAST SALE,May-26-2023,Condo/Co-op,38 Bermuda Colony #10,Fox Lake,IL,60020.0,55000.0,0.0,1.0,...,42.427010,-88.197589,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2657,PAST SALE,May-5-2023,Condo/Co-op,55 Vail Colony #8,Fox Lake,IL,60020.0,60000.0,2.0,1.0,...,42.425521,-88.197562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,PAST SALE,May-22-2023,Condo/Co-op,801 N Mclean Blvd #106,Elgin,IL,60123.0,60000.0,1.0,1.0,...,42.051525,-88.313073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3467,PAST SALE,May-19-2023,Condo/Co-op,222 Madison St #216,Joliet,IL,60435.0,68500.0,1.0,1.0,...,41.527338,-88.134127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3506,PAST SALE,May-31-2023,Townhouse,824 Pin Oak Ln,University Park,IL,60484.0,71000.0,2.0,1.0,...,41.441725,-87.682806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
print(df_filtered['URL'].iloc[3390])

URL    https://www.redfin.com/IL/Wilmington/45-Dinosa...
URL                                                  NaN
Name: 3390, dtype: object


In [11]:
# # Correct the prices, if needed
# df_filtered.at[6,'PRICE']=(742000)
# df_filtered.at[708,'PRICE']=(749000)
# df_filtered.at[858,'PRICE']=(1100000)
# df_filtered.at[49,'PRICE']=(310000)
# df_filtered.at[152,'PRICE']=(650000)

In [12]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [13]:
# # # Corrections, if needed
# df_filtered.at[708,'$/SQUARE FEET']=(749000/959)
# df_filtered.at[858,'$/SQUARE FEET']=(1100000/1251)
# df_filtered.at[6,'$/SQUARE FEET']=(742000/1152)
# df_filtered.at[49,'$/SQUARE FEET']=(310000/1215)
# df_filtered.at[152,'$/SQUARE FEET']=(650000/1487)
# df_filtered.at[682,'$/SQUARE FEET']=(85000/902)

In [14]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
2092,265000.0,4915 W 3rd St,Winthrop Harbor,4.0
3453,186000.0,307 S Raynor Ave,Joliet,24.0
2771,30000.0,2229 Winnebago Rd,Waukegan,39.0
2849,92000.0,154 Barbara Ln,New Lenox,46.0
3506,71000.0,824 Pin Oak Ln,University Park,47.0
890,72000.0,838 Cayuga Trl,Marengo,56.0
1083,130000.0,126 N Mason St,Bensenville,56.0
2115,184900.0,306 Woodlock St,Ingleside,56.0
2722,114400.0,615 May St,Waukegan,57.0
2939,75000.0,317 S Ottawa St,Joliet,58.0


In [15]:
print(df_filtered.URL.iloc[2092])

URL    https://www.redfin.com/IL/Winthrop-Harbor/4915...
URL                                                  NaN
Name: 2092, dtype: object


In [16]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [17]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [18]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [19]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [20]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [21]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [22]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"April 2023 Collar County Single Family Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map

In [23]:
m.save('index.html')

## Summary Info

In [24]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [25]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'Unnamed: 0', 'URL', 'PRICE_AS_CURRENCY',
       'YEAR BUILT DISPLAY', 'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK',
       'FULL_ADDRESS'],
      dtype='object')

In [26]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [27]:
print(df_filtered.loc[df_filtered['PRICE'].idxmin()]['URL'])

URL    https://www.redfin.com/IL/Wilmington/45-Dinosa...
URL                                                  NaN
Name: 3390, dtype: object


In [28]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
Lake Forest, 500 N Green Bay Rd Lake Forest | Price $4,250,000 | $234 psf | Year built: 1895
Least Expensive
Wilmington, 45 Dinosaur Rd Wilmington | Price $29,999 | $61 psf | Year built: 1993
Highest Price Per Square Foot
Elmhurst, 655 W Mary Ct Elmhurst | Price $700,000 | $700,000 psf | Year built: 2001
Lowest Price Per Square Foot
Winthrop Harbor, 4915 W 3rd St Winthrop Harbor | Price $265,000 | $4 psf | Year built: 1972
Newest
Elmhurst, 203 N Walnut St Elmhurst | Price $1,575,000 | $298 psf | Year built: 2023
Oldest
Barrington Area, 511 North Ave Barrington | Price $649,000 | $nan psf | Year built: 1864


In [35]:
df_filtered.sort_values(by='$/SQUARE FEET',ascending=False)

,SOLD DATE,ADDRESS,CITY,PRICE,BEDS,BATHS,LOCATION,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,...,LATITUDE,LONGITUDE,Unnamed: 0,URL,PRICE_AS_CURRENCY,YEAR BUILT DISPLAY,PRICE_SQUARE_FEET_AS_CURRENCY,COLOR,RANK,FULL_ADDRESS
1938,May-15-2023,655 W Mary Ct,Elmhurst,700000.0,0.0,3.0,Elmhurst,2001.0,NaN,700000.0,...,41.920997,-87.962490,NaN,NaN,"$700,000",2001,"$700,000",blue,359,655 W Mary Ct Elmhurst
1764,May-23-2023,845 S Hawthorne Ave,Elmhurst,510000.0,4.0,3.0,Elmhurst,1961.0,NaN,510000.0,...,41.873479,-87.952694,NaN,NaN,"$510,000",1961,"$510,000",blue,778,845 S Hawthorne Ave Elmhurst
1016,May-17-2023,452 E 4th St,Hinsdale,4225000.0,5.0,7.0,Southeast,2006.0,NaN,648.0,...,41.798533,-87.917813,NaN,NaN,"$4,225,000",2006,$648,orange,2,452 E 4th St Hinsdale
2053,May-10-2023,1155 Kelmscott Way #101,Lake Forest,1160000.0,3.0,2.5,Lake Forest,2018.0,NaN,590.0,...,42.260265,-87.843676,NaN,NaN,"$1,160,000",2018,$590,blue,94,1155 Kelmscott Way #101 Lake Forest
1122,May-15-2023,88 Park Ave #3,Clarendon Hills,1340000.0,3.0,2.5,Clarendon Hills,2020.0,NaN,556.0,...,41.795681,-87.953736,NaN,NaN,"$1,340,000",2020,$556,blue,63,88 Park Ave #3 Clarendon Hills
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314,May-25-2023,9A Kingery Quarter Ln #204,Willowbrook,130000.0,1.0,1.0,Hinsdale Point,1981.0,NaN,NaN,...,41.718732,-87.951633,NaN,NaN,"$130,000",1981,$nan,blue,3462,9A Kingery Quarter Ln #204 Willowbrook
1459,May-22-2023,841 N York St #330,Elmhurst,125000.0,1.0,1.0,York Meadows,1975.0,NaN,NaN,...,41.925140,-87.938129,NaN,NaN,"$125,000",1975,$nan,blue,3476,841 N York St #330 Elmhurst
2025,May-12-2023,920 Vose Dr #609,Gurnee,119900.0,1.0,1.0,Heather Ridge,1978.0,NaN,NaN,...,42.344350,-87.953161,NaN,NaN,"$119,900",1978,$nan,blue,3493,920 Vose Dr #609 Gurnee
3531,May-18-2023,308 Dell Park Ave,Lockport,105000.0,3.0,2.0,Homer / Lockport,1941.0,NaN,NaN,...,41.571210,-88.062571,NaN,NaN,"$105,000",1941,$nan,blue,3515,308 Dell Park Ave Lockport


## Time on Market Calculator

In [29]:
# print(df_Current.loc[df_Current['YEAR BUILT'].idxmin()])

In [30]:
from datetime import datetime, timedelta

date1 = datetime(2022, 2, 23) ## List (Earlier) date
date2 = datetime(2023, 4, 11) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

412


## Map URL Snagger

In [31]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [32]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/CollarCounty_single_family_sales_month_ending_may_2023


## Get Summary Data

In [33]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 3558
--------
Total sale price: $1,483,546,990
--------
Median sale price: $350,000
--------
Max sale price: $4,250,000
--------
Min sale price: $29,999
------------------------------------------------
PSF INFO
Max price per square foot: $700,000
--------
Min price per square foot: $4
--------
Median price per square foot: $189
------------------------------------------------
CONDO AGES
Newest building: 2023.0
----------
Oldest building: 1864.0
----------
Average building age: 1983.5718794835007


In [34]:
df_filtered.to_csv('April_CollarCounty.csv')